# Output Parser
- Output Parser는 LLM의 출력을 처리하여 애플리케이션에서 사용할 수 있는 적절한 형식으로 변환하는 역할을 한다. 
  - LLM이 생성한 결과(Raw text)를 분석하여 **특정 정보를 추출**하거나, **원하는 형식으로 재구성**하는 데 사용된다.
- Output parser를 통해 LLM이 응답하는 **비구조적 데이터를 구조화된 데이터로 변환**하여 후속 작업에 적합하게 만드는 데 사용된다.

## 주요 Output Parser
1. **CommaSeparatedListOutputParser**
    - 쉼표로 구분된 텍스트를 리스트로 변환
2. **JsonOutputParser**
    - JSON 형태로 받은 결과를 JSON 형식으로 변환
3. **PydanticOutputParser**
    - JSON 형태로 받은 결과를 Pydantic 모델 객체로 변환
4. **YamlOutputParser**
    - YAML 형태로 받은 응답을 pydantic 모델객체로 변환.
5. **StrOutputParser**
    - 모델의 출력결과를 문자열로 변환
- PydanticOutputParser, JsonOutputParser, YamlOutputParser는 Pydantic을 이용해 schema를 정의하고 이를 이용해 출력을 변환한다.
  
## 메소드
- parse(text:str)
  - LLM이 생성한 응답(text)을 받아 출력 구조에 맞게 변환하여 반환.
- get_format_instructions(): str
  - output parser가 입력받을 형식으로 LLM이 출력(응답) 하도록 하는 프롬프트를 반환한다.
  - LLM에 전송하는 프롬프트에 포함되어 출력 형식을 안내한다.
- [**Runnable**](05_chaing_LECL.ipynb#Runnable)을 상속받아 구현되어 invoke()를 이용해서 parsing 할 수있다.


In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

## StrOutputParser
- 모델(LLM)의 출력 결과를 string으로 변환하여 반환하는 output parser.
- Chat Model은  Message 객체에서 content 속성값을 추출하여 문자열로 반환한다.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
# output parser 
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_template(
    "한국의 {topic} 관련된 속담을 {count}개 알려줘. 목록형식로 작성해줘."
)
model = ChatOpenAI(model="gpt-4o-mini")
# output parser
parser = StrOutputParser()

In [9]:
# 프롬프트 생성
prompt = prompt_template.invoke({"topic":"호랑이", "count":5})
# print(prompt)
# LLM 모델에 요청
res = model.invoke(prompt)
# 응답결과를 확인
print(res)
## parser 를 이용해서 content 만 추출
result = parser.invoke(res)
print(result)

content='1. 호랑이 굴에 가야 호랑이 새끼를 잡는다.\n2. 호랑이 담배 피우던 시절.\n3. 호랑이도 제 말 하면 온다.\n4. 호랑이와 양.\n5. 호랑이의 눈빛.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 31, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None} id='run-9c3ac80a-61a5-431c-869a-504bcb0b0e3f-0' usage_metadata={'input_tokens': 31, 'output_tokens': 61, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
1. 호랑이 굴에 가야 호랑이 새끼를 잡는다.
2. 호랑이 담배 피우던 시절.
3. 호랑이도 제 말 하면 온다.
4. 호랑이와 양.
5. 호랑이의 눈빛.


## CommaSeparatedListOutputParser

- `,`를 구분자로 하는 항목들을 받아서 List로 반환한다.
  - "a,b,c" => ['a','b','c']

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
parser = CommaSeparatedListOutputParser()
txt = "이순신, 유관순, 강감찬, 안중근"
res = parser.invoke(txt)
print(type(res))
print(res)

<class 'list'>
['이순신', '유관순', '강감찬', '안중근']


In [ ]:
# 이 output parser가 변환할 수있는 형식의 문자열을 받기 위한 가이드. 
# 이것을 프롬프트에 추가한다.
print(parser.get_format_instructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [15]:
from langchain.prompts import PromptTemplate

paser = CommaSeparatedListOutputParser()
prompt_template = PromptTemplate(
    template="{subject}의 이름 다섯개를 나열해 주세요.\n{format_instruction}",
    partial_variables={"format_instruction":parser.get_format_instructions()}
)
# partial_variables=dict : 템플릿 변수에 값을 prompt template을 만들면서 넣을 때 사용.
#                          (invoke() 에서 넣지 않고.)
# print(prompt_template.template)
model = ChatOpenAI(model="gpt-4o-mini")

# prompt 생성 - {format_instruction} 변수에는 partial_variables에 설정한 값을 넣는다.
prompt = prompt_template.invoke({"subject":"과일"}) 
# print(prompt)
# llm 요청
res = model.invoke(prompt)

In [23]:
# res.content.split(', ')
print(res.content)
result = parser.invoke(res)
print(result)

사과, 바나나, 오렌지, 포도, 키위
['사과', '바나나', '오렌지', '포도', '키위']


In [24]:
# 입력 -> prompt template -prompt생성-> llm -응답-> parser -> 최종 응답
## chain 생성. (pipeline)
chain = prompt_template | model | parser

result = chain.invoke({"subject":"자동차"})
print(result)

['현대 아반떼', '기아 K5', '토요타 캠리', 'BMW 3시리즈', '메르세데스-벤츠 C-Class']


{'이름': '홍길동'}

## JsonOutputParser

- JSON 형태로 받은 응답을 dictionary로 반환
- JSON 형식을 정하려는 경우 [Pydantic](Pydantic.ipynb)을 이용해 JSON 스키마를 정의하여 JsonOutputParser 생성시 전달한다.
- LLM이 JSON Schema를 따르는 형태로 응답을 하게 하고 그 것을 JsonOutputParser는 Dictionary로 변환한다.



In [30]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
txt = '''{
    "이름":"홍길동",
    "나이":20,
    "취미":["게임", "독서"]
}'''
res = parser.invoke(txt)
print(type(res))
print(res['이름'], res['나이'])
print(parser.get_format_instructions())

<class 'dict'>
홍길동 20
Return a JSON object.


In [33]:
parser = JsonOutputParser()
prompt_template = PromptTemplate(
    template="{name}에 대해서 설명해줘.\n{format_instuction}",
    partial_variables={"format_instuction":parser.get_format_instructions()}
)
model = ChatOpenAI(model="gpt-4o-mini")

prompt = prompt_template.invoke({"name":"아이폰"})
# print(prompt)
res = model.invoke(prompt)

In [ ]:
print(res.content)
print("="*100)

result = parser.invoke(res)
print(type(result))
result

```json
{
  "device": "iPhone",
  "manufacturer": "Apple Inc.",
  "introduction": "The iPhone is a line of smartphones designed and marketed by Apple Inc. It was first introduced in 2007 and has since become one of the most popular smartphones in the world.",
  "features": {
    "operating_system": "iOS",
    "screen_sizes": [
      "4.7 inches",
      "5.5 inches",
      "6.1 inches",
      "6.7 inches"
    ],
    "camera": {
      "front_camera": "12 MP",
      "rear_camera": "up to 48 MP",
      "video_recording": "up to 4K"
    },
    "storage_options": [
      "64 GB",
      "128 GB",
      "256 GB",
      "512 GB",
      "1 TB"
    ],
    "color_options": [
      "Black",
      "White",
      "Red",
      "Green",
      "Blue",
      "Purple",
      "Gold",
      "Silver"
    ],
    "connectivity": [
      "5G",
      "Wi-Fi",
      "Bluetooth",
      "NFC"
    ],
    "special_features": [
      "Face ID",
      "Touch ID",
      "Siri",
      "Apple Pay",
      "App Store"
    ]

{'device': 'iPhone',
 'manufacturer': 'Apple Inc.',
 'introduction': 'The iPhone is a line of smartphones designed and marketed by Apple Inc. It was first introduced in 2007 and has since become one of the most popular smartphones in the world.',
 'features': {'operating_system': 'iOS',
  'screen_sizes': ['4.7 inches', '5.5 inches', '6.1 inches', '6.7 inches'],
  'camera': {'front_camera': '12 MP',
   'rear_camera': 'up to 48 MP',
   'video_recording': 'up to 4K'},
  'storage_options': ['64 GB', '128 GB', '256 GB', '512 GB', '1 TB'],
  'color_options': ['Black',
   'White',
   'Red',
   'Green',
   'Blue',
   'Purple',
   'Gold',
   'Silver'],
  'connectivity': ['5G', 'Wi-Fi', 'Bluetooth', 'NFC'],
  'special_features': ['Face ID',
   'Touch ID',
   'Siri',
   'Apple Pay',
   'App Store']},
 'models': [{'name': 'iPhone 14', 'release_year': 2022},
  {'name': 'iPhone 13', 'release_year': 2021},
  {'name': 'iPhone 12', 'release_year': 2020},
  {'name': 'iPhone SE', 'release_year': 2020}],


In [38]:
result['device']

'iPhone'

In [ ]:
# JSON 문자열의 구조(schema-스키마) 를 정의
## 알고싶은 정보가 무엇인지(json에 어떤 내용을 넣어야 하는지)를 설정.
## pydantic 라이브러리를 이용해서 정의

In [ ]:
# control + ` : 터미널 실행
## pip show pydantic

In [43]:
# 스키마 정의
from pydantic import BaseModel, Field
class Item(BaseModel):
    # 스키마를 class 변수로 정의 - 
    ## 변수명=key. 
    ## type hint로 dict로 변환될때 value의 타입
    name:str = Field(description="제품의 이름")
    info:str = Field(description="제품에 대한 소개 정보")
    price:int = Field(description="제품의 한국에서 가격")

parser = JsonOutputParser(pydantic_object=Item)
print(parser.get_format_instructions())


prompt_template = PromptTemplate(
    template="{name}에 대해서 설명해줘.\n{format_instuction}",
    partial_variables={"format_instuction":parser.get_format_instructions()}
)
model = ChatOpenAI(model="gpt-4o-mini")

prompt = prompt_template.invoke({"name":"갤럭시S 24"})
# print(prompt)
res = model.invoke(prompt)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "제품의 이름", "title": "Name", "type": "string"}, "info": {"description": "제품에 대한 소개 정보", "title": "Info", "type": "string"}, "price": {"description": "제품의 한국에서 가격", "title": "Price", "type": "integer"}}, "required": ["name", "info", "price"]}
```


In [44]:
print(res.content)
result = parser.invoke(res)

print(result)

```json
{
  "name": "갤럭시 S24",
  "info": "갤럭시 S24는 삼성의 최신 스마트폰으로, 향상된 카메라 성능과 뛰어난 배터리 수명을 제공합니다. 혁신적인 디스플레이와 함께 강력한 성능을 자랑하며, 최신 안드로이드 운영체제를 기반으로 합니다.",
  "price": 1200000
}
```
{'name': '갤럭시 S24', 'info': '갤럭시 S24는 삼성의 최신 스마트폰으로, 향상된 카메라 성능과 뛰어난 배터리 수명을 제공합니다. 혁신적인 디스플레이와 함께 강력한 성능을 자랑하며, 최신 안드로이드 운영체제를 기반으로 합니다.', 'price': 1200000}


## PydanticOutputParser

- JSON 형태로 받은 응답을 Pydantic 모델로 변환하여 반환한다.
- 구현은 JsonOutputParser와 동일한데 parsing 결과를 pydantic 모델로 반환한다.

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
class Person(BaseModel):
    name: str = Field(description="사람의 이름")
    yob: int = Field(description="name의 사람이 태어난 년도")
    yod: int = Field(description="name의 사람이 사망한 년도")
    profile:str = Field(description="name의 사람에대한 정보(profile)")

parser = PydanticOutputParser(pydantic_object=Person)
print(parser.get_format_instructions()) 
# JsonOutputParser와 동일한 guide. JSON 응답 -> Pydantic Model 객체로 변환.

prompt_template = PromptTemplate(
    template="{name}에 대해서 설명해주세요.\n{format_instruction}",
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

prompt = prompt_template.invoke({"name":"유관순"})
res = model.invoke(prompt)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "사람의 이름", "title": "Name", "type": "string"}, "yob": {"description": "name의 사람이 태어난 년도", "title": "Yob", "type": "integer"}, "yod": {"description": "name의 사람이 사망한 년도", "title": "Yod", "type": "integer"}, "profile": {"description": "name의 사람에대한 정보(profile)", "title": "Profile", "type": "string"}}, "required": ["name", "yob", "yod", "profile"]}
```


In [49]:
print(res.content)

```json
{
  "name": "유관순",
  "yob": 1902,
  "yod": 1943,
  "profile": "유관순은 한국의 독립운동가로, 3.1 운동에 참여하여 한국의 독립을 위해 싸운 인물입니다. 그녀는 1919년 3월 1일에 시작된 독립운동에서 중요한 역할을 하였으며, 이후 일본 경찰에 체포되어 투옥되었습니다. 유관순은 감옥에서 고문을 당하면서도 독립에 대한 의지를 굽히지 않았고, 그녀의 용기와 희생은 한국 독립운동의 상징으로 여겨지고 있습니다."
}
```


In [50]:
result = parser.invoke(res)
print(type(result))
print(result)

<class '__main__.Person'>
name='유관순' yob=1902 yod=1943 profile='유관순은 한국의 독립운동가로, 3.1 운동에 참여하여 한국의 독립을 위해 싸운 인물입니다. 그녀는 1919년 3월 1일에 시작된 독립운동에서 중요한 역할을 하였으며, 이후 일본 경찰에 체포되어 투옥되었습니다. 유관순은 감옥에서 고문을 당하면서도 독립에 대한 의지를 굽히지 않았고, 그녀의 용기와 희생은 한국 독립운동의 상징으로 여겨지고 있습니다.'


In [51]:
result.name, result.profile

('유관순',
 '유관순은 한국의 독립운동가로, 3.1 운동에 참여하여 한국의 독립을 위해 싸운 인물입니다. 그녀는 1919년 3월 1일에 시작된 독립운동에서 중요한 역할을 하였으며, 이후 일본 경찰에 체포되어 투옥되었습니다. 유관순은 감옥에서 고문을 당하면서도 독립에 대한 의지를 굽히지 않았고, 그녀의 용기와 희생은 한국 독립운동의 상징으로 여겨지고 있습니다.')

## YamlOutputParser
- YAML 형태로 받은 응답을 pydantic Model 객체로 반환한다.

In [ ]:
'''
<info>
  <name>홍길동</name>
  <age>30</age>
  <address>서울</address>
  <hobby>
      <item>독서</item>
      <item>게임</item>
  </hoby>
</info>
'''

In [ ]:
'''
JavaScript Object Notation
{
    "name":"홍길동",
    "age":20
    "hobby":["독서", "게임"]
}
'''

In [ ]:
'''
- name: 홍길동
- age: 20
- hobby:
   - 독서
   - 게임
'''

In [ ]:
from langchain.output_parsers import YamlOutputParser

# pydantic으로 JSON schema 를 정의
class Person(BaseModel):
    name: str = Field(description="사람의 이름")
    yob: int = Field(description="name의 사람이 태어난 년도")
    yod: int = Field(description="name의 사람이 사망한 년도")
    profile:str = Field(description="name의 사람에대한 정보(profile)")

parser = YamlOutputParser(pydantic_object=Person)
print(parser.get_format_instructions()) 
# JsonOutputParser와 동일한 guide. JSON 응답 -> Pydantic Model 객체로 변환.

prompt_template = PromptTemplate(
    template="{name}에 대해서 설명해주세요.\n{format_instruction}",
    partial_variables={"format_instruction": parser.get_format_instructions()}
)

prompt = prompt_template.invoke({"name":"이순신"})

res = model.invoke(prompt)

The output should be formatted as a YAML instance that conforms to the given JSON schema below.

# Examples
## Schema
```
{"title": "Players", "description": "A list of players", "type": "array", "items": {"$ref": "#/definitions/Player"}, "definitions": {"Player": {"title": "Player", "type": "object", "properties": {"name": {"title": "Name", "description": "Player name", "type": "string"}, "avg": {"title": "Avg", "description": "Batting average", "type": "number"}}, "required": ["name", "avg"]}}}
```
## Well formatted instance
```
- name: John Doe
  avg: 0.3
- name: Jane Maxfield
  avg: 1.4
```

## Schema
```
{"properties": {"habit": { "description": "A common daily habit", "type": "string" }, "sustainable_alternative": { "description": "An environmentally friendly alternative to the habit", "type": "string"}}, "required": ["habit", "sustainable_alternative"]}
```
## Well formatted instance
```
habit: Using disposable water bottles for daily hydration.
sustainable_alternative: Switch t

In [55]:
print(res.content)

```
name: 이순신
yob: 1545
yod: 1598
profile: 조선시대의 유명한 군인으로, 임진왜란 동안 일본군과의 전투에서 여러 차례의 승리를 거두며 국가를 지킨 인물입니다.
```


In [56]:
result = parser.invoke(res)
print(type(result))
result

<class '__main__.Person'>


Person(name='이순신', yob=1545, yod=1598, profile='조선시대의 유명한 군인으로, 임진왜란 동안 일본군과의 전투에서 여러 차례의 승리를 거두며 국가를 지킨 인물입니다.')

In [57]:
result.name, result.yod

('이순신', 1598)